## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Norman Wells,CA,65.2820,-126.8329,26.64,93,75,5.75,light snow
1,1,Rikitea,PF,-23.1203,-134.9692,73.49,79,100,18.45,light rain
2,2,Georgetown,MY,5.4112,100.3354,76.98,91,40,5.01,scattered clouds
3,3,Tapes,BR,-30.6733,-51.3958,70.90,61,41,12.17,scattered clouds
4,4,Nikolskoye,RU,59.7035,30.7861,55.02,66,54,12.50,broken clouds
5,5,Port Hardy,CA,50.6996,-127.4199,47.66,86,100,1.81,light rain
6,6,Fort Nelson,CA,58.8053,-122.7002,29.79,93,90,2.30,light snow
7,7,Mehriz,IR,31.5917,54.4318,75.70,17,0,0.63,clear sky
8,8,Ushuaia,AR,-54.8000,-68.3000,49.66,53,40,4.61,scattered clouds
9,9,Punta Arenas,CL,-53.1500,-70.9167,51.91,54,40,28.77,scattered clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) 
                                    & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Georgetown,MY,5.4112,100.3354,76.98,91,40,5.01,scattered clouds
7,7,Mehriz,IR,31.5917,54.4318,75.70,17,0,0.63,clear sky
11,11,Cayenne,GF,4.9333,-52.3333,87.84,83,40,1.99,light rain
14,14,Lamar,US,33.6668,-95.5836,85.64,42,7,4.81,clear sky
18,18,Fairmont,US,39.4851,-80.1426,88.38,66,88,1.01,overcast clouds
31,31,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,69,20,23.02,few clouds
32,32,Kununurra,AU,-15.7667,128.7333,76.98,78,58,4.61,broken clouds
35,35,Victoria,HK,22.2855,114.1577,86.38,78,17,1.01,light rain
38,38,Jamestown,US,42.0970,-79.2353,77.85,69,75,5.99,broken clouds
41,41,Kalmunai,LK,7.4167,81.8167,80.53,76,59,10.11,broken clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                212
City                   212
Country                209
Lat                    212
Lng                    212
Max Temp               212
Humidity               212
Cloudiness             212
Wind Speed             212
Current Description    212
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                209
City                   209
Country                209
Lat                    209
Lng                    209
Max Temp               209
Humidity               209
Cloudiness             209
Wind Speed             209
Current Description    209
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Georgetown,MY,76.98,scattered clouds,5.4112,100.3354,
7,Mehriz,IR,75.70,clear sky,31.5917,54.4318,
11,Cayenne,GF,87.84,light rain,4.9333,-52.3333,
14,Lamar,US,85.64,clear sky,33.6668,-95.5836,
18,Fairmont,US,88.38,overcast clouds,39.4851,-80.1426,
31,Arraial Do Cabo,BR,76.96,few clouds,-22.9661,-42.0278,
32,Kununurra,AU,76.98,broken clouds,-15.7667,128.7333,
35,Victoria,HK,86.38,light rain,22.2855,114.1577,
38,Jamestown,US,77.85,broken clouds,42.0970,-79.2353,
41,Kalmunai,LK,80.53,broken clouds,7.4167,81.8167,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
      lat = row["Lat"]
      lng = row["Lng"]
      params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
      base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
      hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
      try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
      except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Georgetown,MY,76.98,scattered clouds,5.4112,100.3354,Cititel Penang
7,Mehriz,IR,75.70,clear sky,31.5917,54.4318,Mehrgard dwelling/سرای مهرگرد
11,Cayenne,GF,87.84,light rain,4.9333,-52.3333,Hôtel Le Dronmi
14,Lamar,US,85.64,clear sky,33.6668,-95.5836,Kings Inn Paris TX
18,Fairmont,US,88.38,overcast clouds,39.4851,-80.1426,Clarion Inn
31,Arraial Do Cabo,BR,76.96,few clouds,-22.9661,-42.0278,Pousada Porto Praia
32,Kununurra,AU,76.98,broken clouds,-15.7667,128.7333,Hotel Kununurra
35,Victoria,HK,86.38,light rain,22.2855,114.1577,Mini Hotel Central
38,Jamestown,US,77.85,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
41,Kalmunai,LK,80.53,broken clouds,7.4167,81.8167,VS Villa


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df['Hotel Name'] != '')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Georgetown,MY,76.98,scattered clouds,5.4112,100.3354,Cititel Penang
7,Mehriz,IR,75.70,clear sky,31.5917,54.4318,Mehrgard dwelling/سرای مهرگرد
11,Cayenne,GF,87.84,light rain,4.9333,-52.3333,Hôtel Le Dronmi
14,Lamar,US,85.64,clear sky,33.6668,-95.5836,Kings Inn Paris TX
18,Fairmont,US,88.38,overcast clouds,39.4851,-80.1426,Clarion Inn
31,Arraial Do Cabo,BR,76.96,few clouds,-22.9661,-42.0278,Pousada Porto Praia
32,Kununurra,AU,76.98,broken clouds,-15.7667,128.7333,Hotel Kununurra
35,Victoria,HK,86.38,light rain,22.2855,114.1577,Mini Hotel Central
38,Jamestown,US,77.85,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
41,Kalmunai,LK,80.53,broken clouds,7.4167,81.8167,VS Villa


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))